In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.9 MB/s 
     |████████████████████████████████| 596 kB 58.3 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 6.6 MB 57.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install  datasets
from datasets import load_dataset



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 9.6 MB/s 
     |████████████████████████████████| 1.1 MB 54.7 MB/s 
     |████████████████████████████████| 140 kB 50.3 MB/s 
     |████████████████████████████████| 212 kB 60.1 MB/s 
     |████████████████████████████████| 86 kB 3.9 MB/s 
     |████████████████████████████████| 127 kB 15.4 MB/s 
     |████████████████████████████████| 144 kB 49.3 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 271 kB 52.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
ERROR: pip's dependency resolver does not currently take into account all t

# The dataset seems to be hidden behind a `download` link (`dataset_downlolad_url`). 

Invoke the link (`dataset_download_url`) manually to get the zip file instead

In [3]:
(dataset_name, dataset_subset_name) = ("financial_phrasebank", "sentences_allagree")
dataset_downlolad_url = "https://www.researchgate.net/publication/251231364_FinancialPhraseBank-v10/link/0c96051eee4fb1d56e000000/download"
dataset_zipfile_url = "https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip"

download_path = "/content/FinancialPhraseBank-v1.0.zip"

from datasets import load_dataset

# dataset = load_dataset("financial_phrasebank",'sentences_allagree')


The download URL is hidden behind a "download" link
- You must manually visit the URL
- Click on "Download"
- And then upload to the `/content` directory on Colab

In [4]:
import os

if not os.path.exists(download_path):
  print("You must manually go to the URL: ", dataset_zipfile_url, "\n\tdownload the file and upload it to Colab")
  # !wget $dataset_zipfile_url

  

In [5]:
unzipped_dir=download_path.replace(".zip", "")
unzipped_file=os.path.join(unzipped_dir, "Sentences_AllAgree.txt")


if not os.path.exists(unzipped_file):
  ! unzip $download_path

print("Loading: ", unzipped_file)

Archive:  /content/FinancialPhraseBank-v1.0.zip
   creating: FinancialPhraseBank-v1.0/
  inflating: FinancialPhraseBank-v1.0/License.txt  
   creating: __MACOSX/
   creating: __MACOSX/FinancialPhraseBank-v1.0/
  inflating: __MACOSX/FinancialPhraseBank-v1.0/._License.txt  
  inflating: FinancialPhraseBank-v1.0/README.txt  
  inflating: __MACOSX/FinancialPhraseBank-v1.0/._README.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_50Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_66Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_75Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_AllAgree.txt  
Loading:  /content/FinancialPhraseBank-v1.0/Sentences_AllAgree.txt


Unfortunately, the unzipped file is not encoded as utf-8, so `load_dataset` failes when it encounters a non-Unicode character.

Can read it as a CSV file by passing the proper encoding argument and separator.
Then write it back out as a CSV file in "standard" encoding.

In [6]:
import pandas as pd
df = pd.read_csv(unzipped_file, encoding='latin1', delimiter='@', header=None)


In [7]:
unzipped_file_mod = unzipped_file.replace(".txt", "_mod.csv")
(text_hdr, label_hdr) = ("text", "labels")
df.to_csv(unzipped_file_mod, sep="\t", header=[text_hdr, label_hdr], index=None)

In [8]:
raw_datasets = load_dataset("csv", data_files=unzipped_file_mod, delimiter="\t")

Using custom data configuration default-24a82a45209db792


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-24a82a45209db792/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2264
    })
})

In [10]:
raw_datasets["train"][:2]

{'labels': ['neutral', 'positive'],
 'text': ['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
  "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m ."]}

In [11]:
label_to_int = { 'negative': 0, 'neutral': 1, 'positive': 2}
def process_example(example):
  text, label = example[text_hdr], example[label_hdr]

  # Replace label with integer:
  label_int = label_to_int[label]

  return { text_hdr: text, label_hdr: label_int }

In [12]:
example = raw_datasets["train"][0]
print("Raw example: ", example)

print("Processed example: ", process_example(example) )


Raw example:  {'text': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', 'labels': 'neutral'}
Processed example:  {'text': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', 'labels': 1}


In [13]:
processed_dataset = raw_datasets.map(process_example)

  0%|          | 0/2264 [00:00<?, ?ex/s]

In [14]:
processed_dataset["train"][:2]

{'labels': [1, 2],
 'text': ['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
  "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m ."]}

# Alternate (pre-processed) data source

In [15]:
dataset = load_dataset("nickmuchi/financial-classification")


Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Using custom data configuration nickmuchi--distilroberta-finetuned-finclass-abb0f3d5c2987b89


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/nickmuchi___parquet/nickmuchi--distilroberta-finetuned-finclass-abb0f3d5c2987b89/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 506
    })
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 4551
    })
})

In [17]:
dataset["train"][:2]

{'labels': [0, 1],
 'text': ['Finnish airline Finnair is starting the temporary layoffs of cabin crews in February 2010 .',
  'The corresponding increase in the share capital , in total EUR 300,00 was registered in the Finnish Trade Register on May 8 , 2008 .']}

By invoking the model with the `*Model` architecture: we get a model that returns (the sequence of) hidden states.  That is: a model without a head.

Had we invoked it with the `*AutoModelForSequenceClassification` architecture, we get a model with a *binary* classification head.
- But this dataset has *three* classes, so have to design a head with 3 outputs


In [18]:
from transformers import DistilBertTokenizerFast, TFDistilBertModel
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
bert = TFDistilBertModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Let's take a look at the model configuration

In [19]:
from transformers import DistilBertConfig, BertConfig

DistilBertConfig()

DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.19.2",
  "vocab_size": 30522
}

In [20]:
BertConfig()

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

The following code is adapted from the Course example notebook:
- Fine_tune_HuggingFace_model_in_Keras_with_plain_datasets.ipynb

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np

In [22]:
class TextClassificationModel(keras.Model):
  def __init__(self, encoder, train_encoder=True):
    super(TextClassificationModel, self).__init__()
    self.encoder = encoder
    self.encoder.trainable = train_encoder
    self.dropout1 = layers.Dropout(0.1)
    self.dropout2 = layers.Dropout(0.1)
    self.dense1 = layers.Dense(20, activation="relu")
    self.dense2 = layers.Dense(3, activation='softmax')
  
  def call(self, input):
    x = self.encoder(input)
    x = x['last_hidden_state'][:, 0, :]
    x = self.dropout1(x)
    x = self.dense1(x)
    x = self.dropout2(x)
    x = self.dense2(x)
    return x

In [23]:
text_classification_model = TextClassificationModel(bert, train_encoder=True)

In [24]:
len( processed_dataset["train"]["labels"] )

2264

First try:
- place all examples/labels in memory, rather than HF dataset
- then tokenize them and place them in a TF Dataset and free memory

In [25]:
train_texts, train_labels = processed_dataset["train"]["text"], processed_dataset["train"]["labels"]

In [26]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [27]:
len(val_texts)

453

In [28]:
train_encodings = tokenizer(train_texts, truncation=True, padding="max_length", max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding="max_length", max_length=512)

In [29]:
type(train_encodings)

transformers.tokenization_utils_base.BatchEncoding

In [30]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [31]:
type( dict(train_encodings))

dict

In [32]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

## kjp: Let's get an idea of the shape of the model's layers
- We can confirm that
-  the model reduces the sequence of latent representations produced by Bert to a singleton latent representation
- that the latent representation size is 768
  - It is input to the `Dense` layer with 20 units
  - With 768 inputs per unit, plus one bias per unit, we can see that the parameter count for the `Dense` unit that we calculate matches reality
      - 20*768 + 20

In [33]:
text_classification_model(next(iter(train_dataset.batch(4))))

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[0.15416336, 0.53111273, 0.3147239 ],
       [0.10768747, 0.5835469 , 0.30876562],
       [0.17936756, 0.5154524 , 0.30518007],
       [0.15588026, 0.5464263 , 0.29769343]], dtype=float32)>

We can verify 
- from the output that we are outputting probabilities for 3 classes (Negative, Neutral, Positive)
- from the number of parameters to the final `Dense` layer -- 20 inputs * 3 neurons + 2 biases = 63 weights) that there are 3 output classes

In [34]:
text_classification_model.summary()

Model: "text_classification_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_distil_bert_model (TFDis  multiple                 66362880  
 tilBertModel)                                                   
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
 dropout_20 (Dropout)        multiple                  0         
                                                                 
 dense (Dense)               multiple                  15380     
                                                                 
 dense_1 (Dense)             multiple                  63        
                                                                 
Total params: 66,378,323
Trainable params: 66,378,323
Non-trainable params: 0
_____________________________

In [35]:
text_classification_model.compile(
    tf.keras.optimizers.Adam(learning_rate=5e-5), 
    "sparse_categorical_crossentropy", 
    metrics=["accuracy"])


Free up memory

In [36]:
num_train = len(train_texts)

del(train_texts)
del(val_texts)

In [37]:
del(train_encodings)
del(val_encodings)

In [38]:
num_chunks = 4
num_epochs = 1

chunk_size = num_train // num_chunks

print(f"training on {num_train} examples in chunks of size {chunk_size}")

for epoch_num in range(num_epochs):
  for chunk_num in range(num_chunks):
    print(f"Epoch {epoch_num}, chunk {chunk_num}:")
    history = text_classification_model.fit(
      train_dataset.skip(chunk_num * chunk_size).take(chunk_size).shuffle(1000).batch(16), 
      epochs=1, 
      validation_data=val_dataset.take(500).batch(16),
      #callbacks=[tensorboard_callback]
    )

training on 1811 examples in chunks of size 452
Epoch 0, chunk 0:
29/29 [==============================] - 45s 1s/step - loss: 0.6410 - accuracy: 0.7190 - val_loss: 0.3699 - val_accuracy: 0.8389
Epoch 0, chunk 1:
29/29 [==============================] - 33s 1s/step - loss: 0.3074 - accuracy: 0.8872 - val_loss: 0.1719 - val_accuracy: 0.9294
Epoch 0, chunk 2:
29/29 [==============================] - 34s 1s/step - loss: 0.2070 - accuracy: 0.9181 - val_loss: 0.1549 - val_accuracy: 0.9448
Epoch 0, chunk 3:
29/29 [==============================] - 35s 1s/step - loss: 0.1515 - accuracy: 0.9558 - val_loss: 0.1466 - val_accuracy: 0.9470


Evaluate accuracy on validation data

In [39]:
len( val_dataset )

453

In [40]:
batch_size = 16

# kjp: Understand datasets

The model takes a `dict` as argument, **not** an array of examples

The `dict` has keys for
- `input_ids`, `attention_mask`
- the value associated with each key is an array (of length equal to number of examples)

A batch of "examples" is thus a `dict` of arrays, **not** and array of `dict`'s !

`val_dataset` batch is:
- a tuple of length 2
  - features
    - a dict of key/value pairs
      - the values associated with a key is an array of size `batch_size`
  - labels
    - one label per example, hence an array of size `batch_size`

In [41]:
e = next( iter(val_dataset.batch(batch_size)) )
e_features, e_labels = e
e_features

{'attention_mask': <tf.Tensor: shape=(16, 512), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(16, 512), dtype=int32, numpy=
 array([[  101,  1996,  3155, ...,     0,     0,     0],
        [  101, 12849,  2638, ...,     0,     0,     0],
        [  101,  2012,  1996, ...,     0,     0,     0],
        ...,
        [  101, 18347, 15687, ...,     0,     0,     0],
        [  101,  2396,  2080, ...,     0,     0,     0],
        [  101, 13859,  3006, ...,     0,     0,     0]], dtype=int32)>}

In [42]:
e_labels

<tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 1, 2, 0, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1], dtype=int32)>

To **manually** create a batch of examples (features only)
- Need to create a `dict`
- with the same keys
- whose arrays are sub-arrays (of length `batch_size`) of the entire set of examples

In [43]:
b = { k: e_features[k][:batch_size] for k in e_features.keys() }

# Try to predict using our manually created batch
text_classification_model.predict( b )

array([[3.09191773e-06, 9.99106705e-01, 8.90185882e-04],
       [1.49201844e-06, 9.99837637e-01, 1.60814918e-04],
       [7.61613774e-04, 2.40825815e-03, 9.96830165e-01],
       [8.56157243e-01, 3.55494618e-02, 1.08293295e-01],
       [3.29477575e-06, 9.99872446e-01, 1.24172846e-04],
       [9.88782942e-03, 3.75114471e-01, 6.14997685e-01],
       [4.00954951e-03, 9.52520072e-01, 4.34703715e-02],
       [3.35204930e-07, 9.99925375e-01, 7.42670381e-05],
       [5.07080578e-04, 9.92672324e-01, 6.82062423e-03],
       [4.16542889e-06, 9.99551475e-01, 4.44338279e-04],
       [7.36940769e-04, 2.48542894e-03, 9.96777594e-01],
       [1.52760185e-03, 1.37799862e-03, 9.97094393e-01],
       [2.74509375e-06, 9.99795139e-01, 2.02097668e-04],
       [1.68848828e-06, 9.99865055e-01, 1.33276364e-04],
       [1.90492119e-05, 9.98467386e-01, 1.51349604e-03],
       [9.37918740e-05, 9.96497452e-01, 3.40869534e-03]], dtype=float32)

In [44]:
# Compare to predict using the batch created by Dataset operations
# The dataset returns a pair: (features, labels).  Don't need labels to predict so the "[0]" is selecting the features from the pair
text_classification_model.predict( next( iter(val_dataset.batch(batch_size)) )[0] )


array([[3.09191773e-06, 9.99106705e-01, 8.90185882e-04],
       [1.49201844e-06, 9.99837637e-01, 1.60814918e-04],
       [7.61613774e-04, 2.40825815e-03, 9.96830165e-01],
       [8.56157243e-01, 3.55494618e-02, 1.08293295e-01],
       [3.29477575e-06, 9.99872446e-01, 1.24172846e-04],
       [9.88782942e-03, 3.75114471e-01, 6.14997685e-01],
       [4.00954951e-03, 9.52520072e-01, 4.34703715e-02],
       [3.35204930e-07, 9.99925375e-01, 7.42670381e-05],
       [5.07080578e-04, 9.92672324e-01, 6.82062423e-03],
       [4.16542889e-06, 9.99551475e-01, 4.44338279e-04],
       [7.36940769e-04, 2.48542894e-03, 9.96777594e-01],
       [1.52760185e-03, 1.37799862e-03, 9.97094393e-01],
       [2.74509375e-06, 9.99795139e-01, 2.02097668e-04],
       [1.68848828e-06, 9.99865055e-01, 1.33276364e-04],
       [1.90492119e-05, 9.98467386e-01, 1.51349604e-03],
       [9.37918740e-05, 9.96497452e-01, 3.40869534e-03]], dtype=float32)

In [45]:
num_val = 10

# Create batches of size batch_size
# Evaluate on first batch
val_logits = text_classification_model( next( iter(val_dataset.batch(batch_size)) ) ).numpy()
val_preds = np.argmax( val_logits, axis=1)

val_preds

array([1, 1, 2, 0, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1])

In [46]:
from sklearn.metrics import accuracy_score 
accuracy_score( val_labels[:batch_size], val_preds)

1.0

Predict on the **whole** dataset, processed in batches of size batch_size

In [47]:
# Process the WHOLE dataset, in batches of size batch_size

val_logits = text_classification_model.predict( val_dataset.batch(batch_size) )
val_logits.shape



(453, 3)

In [48]:
val_preds = np.argmax(val_logits, axis=1)

In [49]:
accuracy_score( val_labels, val_preds)

0.9470198675496688

Process the WHOLE dataset, using `evaluate` to get the metrics

In [50]:
text_classification_model.evaluate( val_dataset.batch(16) )

29/29 [==============================] - 9s 317ms/step - loss: 0.1466 - accuracy: 0.9470


[0.14657093584537506, 0.9470198750495911]

# We fine-tuned the model above.
- Now freeze the inherited weights and just train the head
- The fine-tuning should give a better result

In [51]:
text_classification_frozen_model = TextClassificationModel(bert, train_encoder=False)

In [52]:
text_classification_frozen_model(next(iter(train_dataset.batch(4))))

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[0.33288074, 0.37685996, 0.29025933],
       [0.2198198 , 0.55186874, 0.22831151],
       [0.12793833, 0.4908345 , 0.3812272 ],
       [0.1393672 , 0.53589606, 0.32473677]], dtype=float32)>

In [53]:
text_classification_frozen_model.summary()

Model: "text_classification_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_distil_bert_model (TFDis  multiple                 66362880  
 tilBertModel)                                                   
                                                                 
 dropout_21 (Dropout)        multiple                  0         
                                                                 
 dropout_22 (Dropout)        multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  15380     
                                                                 
 dense_3 (Dense)             multiple                  63        
                                                                 
Total params: 66,378,323
Trainable params: 15,443
Non-trainable params: 66,362,880
______________________

In [54]:
text_classification_frozen_model.compile(
    tf.keras.optimizers.Adam(learning_rate=5e-5), 
    "sparse_categorical_crossentropy", 
    metrics=["accuracy"])


In [55]:
print(f"training on {num_train} examples in chunks of size {chunk_size}")


for epoch_num in range( 3 * num_epochs):
  for chunk_num in range(num_chunks):
    print(f"Epoch {epoch_num}, chunk {chunk_num}:")
    history = text_classification_frozen_model.fit(
      train_dataset.skip(chunk_num * chunk_size).take(chunk_size).shuffle(1000).batch(16), 
      epochs=1, 
      validation_data=val_dataset.take(500).batch(16),
      #callbacks=[tensorboard_callback]
    )


training on 1811 examples in chunks of size 452
Epoch 0, chunk 0:
29/29 [==============================] - 26s 730ms/step - loss: 0.8579 - accuracy: 0.6195 - val_loss: 0.6603 - val_accuracy: 0.8035
Epoch 0, chunk 1:
29/29 [==============================] - 19s 675ms/step - loss: 0.5880 - accuracy: 0.8186 - val_loss: 0.4586 - val_accuracy: 0.9007
Epoch 0, chunk 2:
29/29 [==============================] - 20s 685ms/step - loss: 0.4141 - accuracy: 0.9004 - val_loss: 0.3412 - val_accuracy: 0.9183
Epoch 0, chunk 3:
29/29 [==============================] - 20s 693ms/step - loss: 0.3235 - accuracy: 0.9204 - val_loss: 0.2724 - val_accuracy: 0.9404
Epoch 1, chunk 0:
29/29 [==============================] - 20s 698ms/step - loss: 0.2915 - accuracy: 0.9204 - val_loss: 0.2369 - val_accuracy: 0.9404
Epoch 1, chunk 1:
29/29 [==============================] - 20s 705ms/step - loss: 0.1980 - accuracy: 0.9580 - val_loss: 0.2146 - val_accuracy: 0.9404
Epoch 1, chunk 2:
29/29 [===========================

In [56]:
text_classification_frozen_model.evaluate( val_dataset.batch(16) )

29/29 [==============================] - 10s 334ms/step - loss: 0.1588 - accuracy: 0.9426


[0.1587677299976349, 0.9426048398017883]